# This notebook describes how to use SQLalchemy ORM

In [1]:
import conn_params
from sqlalchemy import create_engine

from sqlalchemy.orm import sessionmaker
import sqlalchemy as db
from sqlalchemy import Column, Integer, String
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import registry
engine = db.create_engine(
    conn_params.CONNECTION_STRING,
    future=True,
    echo=True
    )

mapper_registry = registry()

@mapper_registry.mapped
class User:
    __tablename__ = "users"
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)
    def __repr__(self):
        return "<User(name='%s', fullname='%s', nickname='%s')>" % (
            self.name,
            self.fullname,
            self.nickname,
        )

getting table representation

In [2]:
User.__table__

Table('users', MetaData(), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('nickname', String(), table=<users>), schema=None)

Getting table name

In [3]:
User.__tablename__

'users'

In [3]:
from sqlalchemy import select

print(select(User.__table__))

SELECT users.id, users.name, users.fullname, users.nickname 
FROM users


Table has been created with usage of mapped_registry


In [4]:
# at this stage there's nothing happend yet - only python object has been created
me = User(name='Ksawery')

adding the table to database

In [5]:
with engine.begin() as conn:
    mapper_registry.metadata.create_all(conn)

2022-10-14 00:14:06,111 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2022-10-14 00:14:06,112 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-10-14 00:14:06,114 INFO sqlalchemy.engine.Engine select current_schema()
2022-10-14 00:14:06,114 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-10-14 00:14:06,115 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2022-10-14 00:14:06,116 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-10-14 00:14:06,116 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-14 00:14:06,117 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-10-14 00:14:06,117 INFO sqlalchemy.engine.Engine [generated in 0.00023s] {'name': 'users'}
2022-10-14 00:14:06,119 INFO sqlalchemy.engine.Engine COMMIT


Working with SQLalchemy Session

In [6]:
Session:sessionmaker = sessionmaker(bind=engine)
session:sessionmaker = Session()

New objects can be placed into session using .add method


In [7]:
session.add(me)

we can look at new collection - there are pending objects

In [8]:
session.new

IdentitySet([<User(name='Ksawery', fullname='None', nickname='None')>])

In this stage the whole commit happends

In [9]:
session.commit()

2022-10-14 00:15:07,006 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-14 00:15:07,008 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname) VALUES (%(name)s, %(fullname)s, %(nickname)s) RETURNING users.id
2022-10-14 00:15:07,008 INFO sqlalchemy.engine.Engine [generated in 0.00051s] {'name': 'Ksawery', 'fullname': None, 'nickname': None}
2022-10-14 00:15:07,018 INFO sqlalchemy.engine.Engine COMMIT


Adding multiple users

In [11]:
another_user = User(name='Jacek')
another_user2 = User(name='Kanapka')

In [12]:
users_to_add = [another_user, another_user2]

In [13]:
session.add_all(users_to_add)

In [14]:
session.new

IdentitySet([<User(name='Jacek', fullname='None', nickname='None')>, <User(name='Kanapka', fullname='None', nickname='None')>])

In [15]:
session.commit()

In [16]:
from sqlalchemy import select

res = session.execute(select(User).where(
    User.name.in_(['Ksawery', 'Jacek'])
    ))

for r in res:
    print(r)

(<User(name='Ksawery', fullname='None', nickname='None')>,)
(<User(name='Jacek', fullname='None', nickname='None')>,)


In [ ]:
# updating

In [20]:
from sqlalchemy import update

update_stmt = update(User).values(
                name='Bilbo').where(User.name == 'Ksawery')

with engine.begin() as conn:
    conn.execute(update_stmt)

In [ ]:
session.execute(select(User).where(User.name.in_(['Ksawery', 'Jacek'])))

In [ ]:
session.new

In [ ]:
query = select(User)

In [ ]:
query

In [ ]:
res = session.execute(query)

for r in res:
    print(r)